# PSPNet 

## Set up
1. download resnet 50 model. This will be our initial model. The code will improve it by using PSPNet architecture. 
2. unzip Camvid data
3. check python version

In [1]:
!wget -O "resnet50_v2.pth" --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA'
!mkdir initmodel && mv resnet50_v2.pth initmodel/

--2023-08-07 20:25:22--  https://docs.google.com/uc?export=download&id=1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA
Resolving docs.google.com (docs.google.com)... 2607:f8b0:4002:c09::64, 2607:f8b0:4002:c09::8b, 2607:f8b0:4002:c09::8a, ...
Connecting to docs.google.com (docs.google.com)|2607:f8b0:4002:c09::64|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nm67gdbkm1gipj06ljdqeq111nd1bije/1691465100000/15543419006824780045/*/1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA?e=download&uuid=edf5bc75-a33d-4c90-a661-b416db5916e2 [following]
--2023-08-07 20:25:27--  https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nm67gdbkm1gipj06ljdqeq111nd1bije/1691465100000/15543419006824780045/*/1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA?e=download&uuid=edf5bc75-a33d-4c90-a661-b416db5916e2
Resolving doc-14-c8-docs.googleusercontent.com (doc-14-c8-docs.googleusercontent.co

In [2]:
!cd Camvid && unzip camvid_semseg11.zip && cd ..

Archive:  camvid_semseg11.zip
   creating: semseg11/
  inflating: semseg11/0016E5_08085_L.png  
  inflating: semseg11/0006R0_f01080_L.png  
  inflating: semseg11/0016E5_00480_L.png  
  inflating: semseg11/0016E5_01350_L.png  
  inflating: semseg11/0016E5_01560_L.png  
  inflating: semseg11/0016E5_01440_L.png  
  inflating: semseg11/0016E5_06150_L.png  
  inflating: semseg11/0016E5_08045_L.png  
  inflating: semseg11/0016E5_00390_L.png  
  inflating: semseg11/0016E5_07680_L.png  
  inflating: semseg11/0016E5_08139_L.png  
  inflating: semseg11/0016E5_08280_L.png  
  inflating: semseg11/0016E5_01110_L.png  
 extracting: semseg11/0016E5_08550_L.png  
  inflating: semseg11/0006R0_f02850_L.png  
  inflating: semseg11/0016E5_05940_L.png  
  inflating: semseg11/0016E5_07230_L.png  
  inflating: semseg11/0016E5_00720_L.png  
  inflating: semseg11/0006R0_f01320_L.png  
  inflating: semseg11/0016E5_08061_L.png  
  inflating: semseg11/0016E5_08019_L.png  
  inflating: semseg11/0016E5_04980_L.png 

In [4]:
!python3 --version

Python 3.8.10


In [5]:
from types import SimpleNamespace

In [7]:
import os
import torch
from torch import nn

# Build Resnet50


In [8]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

In [10]:
""" Args:
    - block: Union[BasicBlock, Bottleneck] indicate if the block is bottleneck or basic block
    - layers: List[int] 
"""

class ResNet(nn.Module):
    def __init__(
        self, block, layers, num_classes: int = 1000, deep_base: bool = True
    ) -> None:
        super(ResNet, self).__init__()
        self.deep_base = deep_base
        if not self.deep_base:
            self.inplanes = 64
            self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.bn1 = nn.BatchNorm2d(64)
        else:
            self.inplanes = 128
            self.conv1 = conv3x3(3, 64, stride=2)
            self.bn1 = nn.BatchNorm2d(64)
            self.conv2 = conv3x3(64, 64)
            self.bn2 = nn.BatchNorm2d(64)
            self.conv3 = conv3x3(64, 128)
            self.bn3 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, planes=64,  blocks=layers[0])
        self.layer2 = self._make_layer(block, planes=128, blocks=layers[1], stride=2)
        self.layer3 = self._make_layer(block, planes=256, blocks=layers[2], stride=2)
        self.layer4 = self._make_layer(block, planes=512, blocks=layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def _make_layer(self, block, planes: int, blocks: int, stride: int = 1):
        """
        Args:
            block: Union[BasicBlock, Bottleneck] structure of fundamental block of layers that is repeated
            planes
            blocks: number of times the block module is repeated sequentially
            stride: stride of conv layers

        Returns:
        """
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

In [12]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample = None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [13]:
def resnet50(pretrained: bool = False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        # model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
        model_path = "./initmodel/resnet50_v2.pth"
        model.load_state_dict(torch.load(model_path), strict=False)
    return model

## Build PSPNet
The final feature map size is 1/8 of the input image.
### 1) Set up the model with ResNet50
The model will have 5 layers which got from pretrained-deep-based ResNet50. Layer0 will be sequential of conv1 -> bn1 -> relu -> conv2 -> bn2 -> relu -> conv3 -> bn3 -> relu -> maxpool.
